In [1]:
import urllib.request as request
import json
import boto3
import pandas as pd

In [ ]:
#reference code for Lambda. ignore all following cells
def lambda_handler(event, context):
    #used to parse the event object
    #print(event)
    #print(event['Records'][0]['s3']['bucket']['name'])
    #dynamic bucket name used for upload later
    bucket_name = event['Records'][0]['s3']['bucket']['name']
    #read the timestamd data to combine into a filename to load 
    s3_read = boto3.resource('s3')
    #bucketname = 'capstone-team-uk-2-data-dump-bucket'
    itemname = 'raw-zone/timestamp_file_covid.txt'
    obj = s3_read.Object(bucket_name, itemname)
    body_timestamp = obj.get()['Body'].read()
    timestamp = body_timestamp.decode()
    print(timestamp)
    covid_filename = 'covid-data-'+timestamp+'.json'
    #load from s3 raw zone
    s3_read = boto3.resource('s3')
    #bucketname = 'capstone-team-uk-2-data-dump-bucket'
    itemname = 'raw-zone/'+covid_filename
    obj = s3_read.Object(bucket_name, itemname)
    body = obj.get()['Body'].read()
    data = json.loads(body)
    json_simple = data['features']
    new_list_of_dicts = [x['attributes'] for x in json_simple]
    data_f = pd.DataFrame(new_list_of_dicts)
    data_f.index.name = 'id'
    output_csv = pd.DataFrame.to_csv(data_f)
    output_csv = data_f.to_csv('/tmp/covid-data.csv')
    s3_upload = boto3.client('s3')
    with open('/tmp/covid-data.csv', "rb") as f:
        s3_upload.upload_fileobj(f, bucket_name, "covid-data/covid-data"+timestamp+".csv")
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }


In [2]:
#read the timestamd data to combine into a filename to load 
s3_read = boto3.resource('s3')
bucketname = 'capstone-team-uk-data-dump-bucket'
itemname = 'raw-zone/timestamp_file_covid.txt'
obj = s3_read.Object(bucketname, itemname)
body_timestamp = obj.get()['Body'].read()

In [3]:
timestamp = body_timestamp.decode()

In [4]:
timestamp

'2020-07-24_12_37_PM'

In [5]:
covid_filename = 'covid-data-'+timestamp+'.json'

In [7]:
#load from s3 raw zone
s3_read = boto3.resource('s3')
bucketname = 'capstone-team-uk-data-dump-bucket'
itemname = 'raw-zone/'+covid_filename
obj = s3_read.Object(bucketname, itemname)
body = obj.get()['Body'].read()

In [9]:
data = json.loads(body)

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [10]:
json_simple = data['features']

In [11]:
json_simple

[{'attributes': {'OBJECTID': 1,
   'FULL_': 'Saguache County',
   'GEOID': '08109',
   'LABEL': 'Saguache',
   'STATEFP': '08',
   'COUNTY': 'SAGUACHE',
   'COUNTYFP': '109',
   'County_Pos_Cases': 106,
   'County_Population': 6837,
   'County_Rate_Per_100_000': 1550.39,
   'County_Pos_Cases_Yesterday': 106,
   'County_Pos_Cases_Change': 0,
   'County_Deaths': 2,
   'County_Deaths_Yesterday': 2,
   'County_Deaths_Change': 0,
   'State_Pos_Cases': 42314,
   'State_Population': 5694287,
   'State_Rate_Per_100000': 743.1,
   'State_Deaths': 1786,
   'State_CDC_Deaths': 1645,
   'State_Number_Hospitalizations': 6149,
   'State_Number_Tested': 467726,
   'State_Test_Encounters': 594731,
   'State_Number_of_Counties_Pos': 63,
   'State_Number_of_Outbreaks': 452,
   'Data_Source': 'Colorado Department of Public Health and Environment',
   'Date_Data_Last_Updated': 'Data through July 22, 2020',
   'Shape__Area': 0.84314903059385,
   'Shape__Length': 4.39184364534676}},
 {'attributes': {'OBJECT

In [12]:
new_list_of_dicts = [
    x['attributes'] for x in json_simple
]

In [14]:
new_list_of_dicts

[{'OBJECTID': 1,
  'FULL_': 'Saguache County',
  'GEOID': '08109',
  'LABEL': 'Saguache',
  'STATEFP': '08',
  'COUNTY': 'SAGUACHE',
  'COUNTYFP': '109',
  'County_Pos_Cases': 106,
  'County_Population': 6837,
  'County_Rate_Per_100_000': 1550.39,
  'County_Pos_Cases_Yesterday': 106,
  'County_Pos_Cases_Change': 0,
  'County_Deaths': 2,
  'County_Deaths_Yesterday': 2,
  'County_Deaths_Change': 0,
  'State_Pos_Cases': 42314,
  'State_Population': 5694287,
  'State_Rate_Per_100000': 743.1,
  'State_Deaths': 1786,
  'State_CDC_Deaths': 1645,
  'State_Number_Hospitalizations': 6149,
  'State_Number_Tested': 467726,
  'State_Test_Encounters': 594731,
  'State_Number_of_Counties_Pos': 63,
  'State_Number_of_Outbreaks': 452,
  'Data_Source': 'Colorado Department of Public Health and Environment',
  'Date_Data_Last_Updated': 'Data through July 22, 2020',
  'Shape__Area': 0.84314903059385,
  'Shape__Length': 4.39184364534676},
 {'OBJECTID': 2,
  'FULL_': 'Sedgwick County',
  'GEOID': '08115',
 

In [15]:
data_f = pd.DataFrame(new_list_of_dicts)

In [16]:
data_f

,OBJECTID,FULL_,GEOID,LABEL,STATEFP,COUNTY,COUNTYFP,County_Pos_Cases,County_Population,County_Rate_Per_100_000,...,State_CDC_Deaths,State_Number_Hospitalizations,State_Number_Tested,State_Test_Encounters,State_Number_of_Counties_Pos,State_Number_of_Outbreaks,Data_Source,Date_Data_Last_Updated,Shape__Area,Shape__Length
0,1,Saguache County,08109,Saguache,08,SAGUACHE,109,106.0,6837.0,1550.39,...,1645,6149,467726,594731,63,452,Colorado Department of Public Health and Envir...,"Data through July 22, 2020",0.843149,4.391844
1,2,Sedgwick County,08115,Sedgwick,08,SEDGWICK,115,11.0,2275.0,483.52,...,1645,6149,467726,594731,63,452,Colorado Department of Public Health and Envir...,"Data through July 22, 2020",0.152011,1.707546
2,3,Cheyenne County,08017,Cheyenne,08,CHEYENNE,017,7.0,1862.0,375.94,...,1645,6149,467726,594731,63,452,Colorado Department of Public Health and Envir...,"Data through July 22, 2020",0.478628,3.111271
3,4,Custer County,08027,Custer,08,CUSTER,027,10.0,4928.0,202.92,...,1645,6149,467726,594731,63,452,Colorado Department of Public Health and Envir...,"Data through July 22, 2020",0.196859,2.364503
4,5,La Plata County,08067,La Plata,08,LA PLATA,067,180.0,56403.0,319.13,...,1645,6149,467726,594731,63,452,Colorado Department of Public Health and Envir...,"Data through July 22, 2020",0.447278,3.022820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,63,Las Animas County,08071,Las Animas,08,LAS ANIMAS,071,13.0,14491.0,89.71,...,1645,6149,467726,594731,63,452,Colorado Department of Public Health and Envir...,"Data through July 22, 2020",1.257277,5.537752
63,64,Grand County,08049,Grand,08,GRAND,049,39.0,15474.0,252.04,...,1645,6149,467726,594731,63,452,Colorado Department of Public Health and Envir...,"Data through July 22, 2020",0.511446,4.130965
64,65,OUT OF STATE,None,OUT OF STATE,08,OUT OF STATE,None,NaN,NaN,NaN,...,1645,6149,467726,594731,63,452,Colorado Department of Public Health and Envir...,"Data through July 22, 2020",NaN,NaN
65,66,UNKNOWN,None,UNKNOWN,08,UNKNOWN,None,NaN,NaN,NaN,...,1645,6149,467726,594731,63,452,Colorado Department of Public Health and Envir...,"Data through July 22, 2020",NaN,NaN


In [17]:
data_f.index.name = 'id'

In [18]:
output_csv = pd.DataFrame.to_csv(data_f)

In [33]:
output_csv = data_f.to_csv('covid-data.csv')

In [34]:
s3_upload = boto3.client('s3')
with open('covid-data.csv', "rb") as f:
    s3_upload.upload_fileobj(f, "capstone-team-uk-data-dump-bucket", "covid-data/covid-data"+timestamp+".csv")